In [ ]:
!pip install langchain
!pip install beautifulsoup4
!pip install dotenv
!pip install langchain_community

In [ ]:
#create webloader to extract data from web pages using langchain
from langchain.document_loaders import WebBaseLoader
import os
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import requests
import bs4
import pandas as pd

In [ ]:
def extract_info(item):
    date=item.find(class_='views-field-created')
    link=item.find(class_='views-field-title').find('a')['href']
    page_response = requests.get(link)
    page_soup = BeautifulSoup(page_response.content, 'html.parser')
    content = page_soup.find('div', class_='field field-name-body field-type-text-with-summary field-label-hidden')
    title = page_soup.find('h1').text.strip()
    content_text = content.text.strip() if content else ''
    data_point = {'Date': date.text.strip() if date else 'N/A', 'Content': 'Title:' + title + '\nArticle:' + content_text}
    return data_point

def get_articles(URL):
    response = requests.get(URL)
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table')
    list_data = []
    for row in table.find_all('tr'):
        list_data.append(extract_info(row))
    return pd.DataFrame(list_data)

def get_latest_articles(URL):
    response = requests.get(URL)
    soup = BeautifulSoup(response.content, 'html.parser')
    latest=soup.find(id='views-bootstrap-grid-1').find_all(class_='field-content')
    list_data = []
    for row in latest:
        list_data.append(extract_info(row))
    return pd.DataFrame(list_data)

def get_bullionvault_articles():
    df_latest = get_latest_articles(URL)
    df=get_articles(URL)
    df = pd.concat([df_latest, df], ignore_index=True)
    for i in range(1, 11):
        next_url = f"{URL}/latest-articles?page={i}"
        df_next = get_articles(next_url)
        df = pd.concat([df, df_next], ignore_index=True)
    return df

URL="https://www.bullionvault.com/gold-news"
bullion_df=get_bullionvault_articles()
bullion_df['Date'] = pd.to_datetime(bullion_df['Date'], errors='coerce')

NameError: name 'df' is not defined